# Validacao dos Resultados - C3

In [52]:

from dotenv import load_dotenv
import os
import sys
import json
import pandas as pd
import time
import datetime
load_dotenv()

experiment_path = '../..'

path = os.path.abspath('')
module_path = os.path.join(path, experiment_path)
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"/functions")


from dataset_utils import DatasetEvaluator

/Users/moon/Documents/text_to_sql_byLLM/experiments/C3/../..


In [53]:
DEFAULT_PATH = "results/"
EXPERIMENT = "C3"
TARGET_PATH = f"evaluator_results/{EXPERIMENT}_results.csv"
TARGET_PATH_FK = f"evaluator_results/{EXPERIMENT}_foreign_key_results.csv"

Funções úteis

In [54]:
def save_queries(queries, FILE_NAME_PATH):
    data = {"queries":queries}
    with open(FILE_NAME_PATH, "w") as arquivo_json:
        json.dump(data, arquivo_json, indent=4) 

def read_queries(FILE_NAME_PATH):
    with open(FILE_NAME_PATH, encoding="utf-8", errors="ignore") as json_data:
        data = json.load(json_data, strict=False)
    queries = data["queries"]
    return queries

def get_schema(FILE_NAME_PATH):
    if "mondial_gpt" in FILE_NAME_PATH:
        return "mondial_gpt"
    elif "mondial" in FILE_NAME_PATH:
        return "mondial"
    return ""

def get_count_query_type(queries):
    count_types = {}
    for item in queries:
        type = item["type"]
        if type in count_types:
            count_types[type] += 1
        else:
            count_types[type] = 1
    count_types['total'] = len(queries)
    return count_types

def get_dataset_evaluator(FILE_NAME_PATH):
    SCHEMA = get_schema(FILE_NAME_PATH)  
    PREFIX = SCHEMA  
    if SCHEMA == "mondial_gpt":
        PREFIX = "mondial"
    print(PREFIX)
    print(SCHEMA)
    dataset_eval = DatasetEvaluator(dataset_file_path=f"../../datasets/{PREFIX}/{PREFIX}_dataset.json",
                                dataset_tables_path=f"../../datasets/{PREFIX}/result_tables/",
                                db_connection_file=f"../../datasets/{SCHEMA}_db_connection.json",
                                dataset_name=SCHEMA)
    return dataset_eval

def get_files_in_folder():
    files = []
    for r, d, f in os.walk(DEFAULT_PATH):
        for file in f:
            if '.json' in file and 'erro' not in file:
                files.append(os.path.join(r, file))
    return sorted(files)

def evaluate_query(queries, dataset_eval, FILE_NAME_PATH):
    result = {'simple': 0, 'medium': 0, 'complex': 0, 'total': 0}

    for q in queries:
        evaluate_result = False
        q['similarity'] = 0
        try:
            if q["query_string"].strip() != "":
                evaluate_result, similarity, column_matching_index = dataset_eval.evaluate_dataset_query(q["query_string"], q["id"], query_type="sql")
                q['column_matching_index'] = column_matching_index
                q['similarity']=similarity
            
        except Exception as e:
            print(str(e))
            
        q['result'] = evaluate_result
        
        if evaluate_result==True:
            result['total'] += 1
            result[q["type"]] += 1
        
    save_queries(queries, FILE_NAME_PATH)   
    return result

def get_file_name(FILE_NAME_PATH):
    parts = FILE_NAME_PATH.split("/")
    last_element = parts[-1]
    file_name = last_element.replace(".json", "")
    return file_name

def get_experiment_name(filename):
    schema = get_schema(filename).upper()
    experiment_name = f"{EXPERIMENT} - {schema}"
    if "chatgpt4" in filename:
        experiment_name += " - GPT-4"
    elif "chatgpt_" in filename:
        experiment_name += " - GPT-3.5"
    elif "llama" in filename:
        experiment_name += " - LLAMA"
    
    if "turbo" in filename:
        experiment_name += " - Turbo"
        
    if "instances" in filename:
        experiment_name += " - Passing Database Rows"
    elif "code_representation" in filename:
        experiment_name += " - Code Representation"
    if "_fk" in filename:
        experiment_name += " - FK"       
    return experiment_name

def compute_cost(filename):
    queries = read_queries(filename)
    total_costs = 0
    total_tokens = 0
    input_tokens = 0
    output_tokens = 0
    #total_token
    for q in queries:
        data = q.get('token_usage', None)
        if data is not None:
            for key, track_token in data.items():
                total_tokens += track_token.get('total_tokens', 0)
                total_costs += track_token.get('total_cost', 0)
                input_tokens += track_token.get('prompt_tokens', 0)
                output_tokens += track_token.get('completion_tokens', 0)
    print(total_tokens)
    print(round(total_costs, 2))

    return total_tokens, round(total_costs, 2), input_tokens, output_tokens

def compute_cost_specific(filename, specific_trackin):
    queries = read_queries(filename)
    specific_info = 0
    for q in queries:
        data = q.get('token_usage', None)
        if data is not None:
            for key, track_token in data.items():
                specific_info += track_token.get(specific_trackin, 0)
    return specific_info

def compute_time(filename):
    queries = read_queries(filename)
    total_time = 0
    for q in queries:
        total_time += q.get('time', 0)
    print(total_time)
    return str(datetime.timedelta(seconds=round(total_time)))

In [55]:
def run(result_files = [], results = [], compute_costs = True):
    
    if result_files == [] :
        result_files = get_files_in_folder()
    else:
        result_files = sorted(result_files)
        
    
    for file in result_files:
        queries = read_queries(file)
        dataset_eval = get_dataset_evaluator(file)
        count_query_type = get_count_query_type(queries)
        result = evaluate_query(queries, dataset_eval, file)
        
        overall_by_type = {}
        for item in result:
            overall_by_type[f"overall_{item}"] = round(result[item] / count_query_type[item], 4)
        
        result['experiment'] = get_file_name(file)
        result = {**result, **overall_by_type}
        
        result['total_tokens'], result['total_cost'], result['average_tokens_by_query'] = 0, 0, 0
        if compute_costs:
            result['total_tokens'], result['total_cost'], result['input_tokens'], result['output_tokens'] = compute_cost(file)
        result['total_time'] = compute_time(file)
        results.append(result)
        time.sleep(5)
    return results


In [56]:
compute_cost('results/5_c3_queries_chatgpt_mondial_gpt_fk.json')

626143
1.16


(626143, 1.16, 177696, 448447)

In [57]:
compute_cost_specific('results/5_c3_queries_chatgpt_mondial_gpt_fk.json', 'prompt_tokens')

177696

In [58]:
compute_cost_specific('results/5_c3_queries_chatgpt_mondial_gpt_fk.json', 'completion_tokens')

448447

# Validação com os Esquemas com Foreign Keys

In [59]:
results_fk = run([
 'results/5_c3_queries_chatgpt_mondial_gpt_fk.json', 'results/6_c3_queries_chatgpt4_mondial_gpt_fk.json'], [])

mondial
mondial_gpt
{'DB_HOST': 'localhost', 'DB_PORT': '1521', 'DB_USER_NAME': 'MONDIAL', 'DB_PASS': 'oraclee', 'DB_NAME': 'XE', 'SQL_DRIVER': 'oracle+oracledb', 'SERVICE_NAME': 'XE', 'SCHEMA': 'MONDIAL_GPT', 'KEYWORD_SEARCH_API_URL': ''}
SELECT max(area) FROM mondial_country where name  = 'Thailand'


/Users/moon/Documents/text_to_sql_byLLM/experiments/C3/../../functions/dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
/Users/moon/Documents/text_to_sql_byLLM/experiments/C3/../../functions/dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
/Users/moon/Documents/text_to_sql_byLLM/experiments/C3/../../functions/dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlche

Execution failed on sql 'SELECT max(area) FROM mondial_country where name  = 'Thailand'': ORA-00942: table or view does not exist
Help: https://docs.oracle.com/error-help/db/ora-00942/
SELECT name FROM mondial_province where area > 10000
Execution failed on sql 'SELECT name FROM mondial_province where area > 10000': ORA-00942: table or view does not exist
Help: https://docs.oracle.com/error-help/db/ora-00942/
SELECT mondial_language.name FROM mondial_language INNER JOIN mondial_country ON mondial_language.country = mondial_country.code WHERE mondial_country.name = 'Poland'
Execution failed on sql 'SELECT mondial_language.name FROM mondial_language INNER JOIN mondial_country ON mondial_language.country = mondial_country.code WHERE mondial_country.name = 'Poland'': ORA-00942: table or view does not exist
Help: https://docs.oracle.com/error-help/db/ora-00942/
SELECT depth FROM mondial_lake where name LIKE '%Lake Kariba%'
Execution failed on sql 'SELECT depth FROM mondial_lake where name L

/Users/moon/Documents/text_to_sql_byLLM/experiments/C3/../../functions/dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
/Users/moon/Documents/text_to_sql_byLLM/experiments/C3/../../functions/dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
/Users/moon/Documents/text_to_sql_byLLM/experiments/C3/../../functions/dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlche

Execution failed on sql 'SELECT name
FROM mondial_desert
WHERE area > 10000': ORA-00942: table or view does not exist
Help: https://docs.oracle.com/error-help/db/ora-00942/
SELECT island.name 
FROM island 
JOIN lakeonisland ON island.name = lakeonisland.island 
JOIN lake ON lake.name = lakeonisland.lake
WHERE lake.elevation > 100;
SELECT island.name FROM island JOIN lakeonisland ON island.name = lakeonisland.island JOIN lake ON lake.name = lakeonisland.lake WHERE lake.elevation > 100;
True cols: Index(['NAME'], dtype='object')
Adj. True cols: Index(['NAME'], dtype='object')
Predicted cols: Index(['NAME'], dtype='object')
Adj. Predicted cols: Index(['NAME'], dtype='object')
True: query matches sql (Similarity: 1.0, column match: 1.0)
SELECT moi.mountain
FROM mondial_mountainonisland moi
INNER JOIN mondial_island isl
ON moi.island = isl.name
WHERE isl.area > 100000
Execution failed on sql 'SELECT moi.mountain
FROM mondial_mountainonisland moi
INNER JOIN mondial_island isl
ON moi.island =

/Users/moon/Documents/text_to_sql_byLLM/experiments/C3/../../functions/dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
/Users/moon/Documents/text_to_sql_byLLM/experiments/C3/../../functions/dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
/Users/moon/Documents/text_to_sql_byLLM/experiments/C3/../../functions/dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlche

In [60]:
df_fk = pd.DataFrame(results_fk, columns=['experiment', 'simple', 'medium', 'complex', 'total', 'overall_simple', 'overall_medium', 'overall_complex', 'overall_total','input_tokens', 'output_tokens', 'total_tokens','total_cost', 'total_time'])
df_fk['experiment'] = df_fk['experiment'].apply(get_experiment_name)
df_fk = df_fk.sort_values(by='overall_total', ascending=False)
df_fk

,experiment,simple,medium,complex,total,overall_simple,overall_medium,overall_complex,overall_total,input_tokens,output_tokens,total_tokens,total_cost,total_time
1,C3 - MONDIAL_GPT - GPT-4 - FK,1,2,1,4,0.0303,0.0606,0.0294,0.04,153705,426937,580642,30.23,1:20:06
0,C3 - MONDIAL_GPT - GPT-3.5 - FK,1,0,0,1,0.0303,0.0000,0.0000,0.01,177696,448447,626143,1.16,2:44:42


In [61]:
df_fk.to_csv(TARGET_PATH_FK)  